In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from tqdm import tqdm

# Ticker symbols for the 12 stocks
tickers = [
    "HDFCBANK.NS", "ICICIBANK.NS", "SBIN.NS", "KOTAKBANK.NS", "AXISBANK.NS",
    "PNB.NS", "INDUSINDBK.NS", "BANDHANBNK.NS", "LICHSGFIN.NS", "HDFCLIFE.NS",
    "SBILIFE.NS", "UTIAMC.NS"
]

# Download historical OHLCV from 2014-01-01 to 2023-12-31
def download_data(ticker):
    data = yf.download(ticker, start="2014-01-01", end="2023-12-31")
    data = data.dropna()
    return data

stock_data = {ticker: download_data(ticker) for ticker in tqdm(tickers)}

  0%|          | 0/12 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 12/12 [00:05<00:00,  2.09it/s]


In [2]:
def compute_features(df):
    df = df.copy()

    # Ensure Close, High, Low are Series (1D)
    close = df["Close"].squeeze()
    high = df["High"].squeeze()
    low = df["Low"].squeeze()
    volume = df["Volume"].squeeze()

    print(f"Close dtype: {type(close)}, shape: {close.shape}")

    # Daily return
    df["Return"] = close.pct_change()

    # 30-day rolling volatility (target)
    df["Volatility"] = df["Return"].rolling(window=30).std()

    # RSI (14 days)
    df["RSI"] = RSIIndicator(close=close, window=14).rsi()

    # Momentum (5 days)
    df["MOM"] = close - close.shift(5)

    # OBV
    df["OBV"] = (np.sign(close.diff()) * volume).fillna(0).cumsum()

    # MACD
    macd = MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD_LINE"] = macd.macd()
    df["MACD_SIGNAL"] = macd.macd_signal()
    df["MACD_HIST"] = macd.macd_diff()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    df["STO_K"] = stoch.stoch()           # formerly %K
    df["STO_D"] = stoch.stoch_signal()    # formerly %D

    # Lagged volatilities (t-1 to t-6)
    for i in range(1, 7):
        df[f"Vol_t_{i}"] = df["Volatility"].shift(i)

    # Volatility t+1 (our target)
    df["Vol_target"] = df["Volatility"].shift(-1)

    # Drop rows with NaNs
    df = df.dropna()

    return df

In [3]:
import os
import pickle

feature_data = {}
for ticker in tqdm(tickers):
    feature_data[ticker] = compute_features(stock_data[ticker])

feature_data_path = "India_feature_data.pkl"

if os.path.exists(feature_data_path):
    print("📦 Loading saved feature data from India_feature_data.pkl...")
    with open(feature_data_path, "rb") as f:
        feature_data = pickle.load(f)
else:
    print("⚙️ Computing feature data...")
    feature_data = {ticker: compute_features(stock_data[ticker]) for ticker in tqdm(tickers)}
    with open(feature_data_path, "wb") as f:
        pickle.dump(feature_data, f)
    print("💾 Saved feature data to India_feature_data.pkl")

100%|██████████| 12/12 [00:00<00:00, 80.65it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1423,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1511,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1543,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (797,)
⚙️ Computing feature data...


  0%|          | 0/12 [00:00<?, ?it/s]

Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)


100%|██████████| 12/12 [00:00<00:00, 82.97it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1423,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (2465,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1511,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (1543,)
Close dtype: <class 'pandas.core.series.Series'>, shape: (797,)
💾 Saved feature data to India_feature_data.pkl


In [4]:
from arch import arch_model
import warnings

def add_garch_predictions(df, ticker=None, verbose=True):
    df = df.copy()
    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # recommended by arch package

    if verbose:
        print(f"\n🔍 GARCH modeling for {ticker} — total points: {len(returns)}")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor  # rescale

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = arch_model(train_window, vol='Garch', p=1, q=1, dist='normal', rescale=False)
                model_fit = model.fit(disp="off")
                forecast = model_fit.forecast(horizon=1)
                pred_vol_scaled = np.sqrt(forecast.variance.values[-1][0])
                pred_vol = pred_vol_scaled / scale_factor  # unscale
        except Exception as e:
            if verbose:
                print(f"⚠️ Failed at i={i} — {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → Index {i} | Pred Vol (unscaled): {pred_vol:.5f}")

    full_preds = [np.nan] * window_size + preds
    df["GARCH_pred"] = full_preds

    before = len(df)
    df = df.dropna()
    after = len(df)

    if verbose:
        print(f"✅ Done {ticker} | Rows dropped: {before - after} | Final: {after} rows")

    return df

In [5]:
# === Try loading precomputed garch_data from disk ===
garch_data_path = "India_garch_data.pkl"

if os.path.exists(garch_data_path):
    print("📦 Loading saved GARCH data from India_garch_data.pkl...")
    with open(garch_data_path, "rb") as f:
        garch_data = pickle.load(f)
    print("✅ Loaded GARCH data successfully!")
else:
    print("⚙️ Computing GARCH data from scratch...")
    garch_data = {}
    for ticker in tickers:
        print(f"\n====================== {ticker} ======================")
        garch_data[ticker] = add_garch_predictions(feature_data[ticker], ticker=ticker)

    # Save to disk
    with open(garch_data_path, "wb") as f:
        pickle.dump(garch_data, f)
    print("💾 Saved GARCH data to India_garch_data.pkl")

⚙️ Computing GARCH data from scratch...

====================== HDFCBANK.NS ======================

🔍 GARCH modeling for HDFCBANK.NS — total points: 2428
  → Index 500 | Pred Vol (unscaled): 0.01166
  → Index 750 | Pred Vol (unscaled): 0.00910
  → Index 1000 | Pred Vol (unscaled): 0.00954
  → Index 1250 | Pred Vol (unscaled): 0.00975
  → Index 1500 | Pred Vol (unscaled): 0.05400
  → Index 1750 | Pred Vol (unscaled): 0.02548
  → Index 2000 | Pred Vol (unscaled): 0.03073
  → Index 2250 | Pred Vol (unscaled): 0.01220
✅ Done HDFCBANK.NS | Rows dropped: 500 | Final: 1928 rows

====================== ICICIBANK.NS ======================

🔍 GARCH modeling for ICICIBANK.NS — total points: 2428
  → Index 500 | Pred Vol (unscaled): 0.01937
  → Index 750 | Pred Vol (unscaled): 0.02107
  → Index 1000 | Pred Vol (unscaled): 0.01896
  → Index 1250 | Pred Vol (unscaled): 0.01590
  → Index 1500 | Pred Vol (unscaled): 0.06086
  → Index 1750 | Pred Vol (unscaled): 0.02125
  → Index 2000 | Pred Vol (unsca

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def evaluate(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_ml_models_baseline(df, ticker="TICKER"):
    print(f"\n📈 Training ML models for {ticker}...")

    # Feature and target selection
    features = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D',
        'Vol_t_1', 'Vol_t_2', 'Vol_t_3', 'Vol_t_4', 'Vol_t_5', 'Vol_t_6'
    ]

    X = df[features].copy()
    # Sanitize column names just in case LightGBM is sensitive
    X.columns = [str(col).replace("-", "_").replace("%", "PCT").replace(".", "_DOT_") for col in X.columns]

    y = df["Vol_target"]

    # Static train-test split (same as paper: 2014–2020 train, 2021–2023 test)
    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        #"LightGBM": LGBMRegressor(),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = evaluate(y_test, y_pred)
        results[name] = metrics
        print(f"✅ {name} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return results

/shared/home/ads066/.local/lib/python3.11/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mse),
        "MSE": mse,
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_all_stocks_ml_baseline(garch_data_dict, results_path="India_ml_baseline_results.csv"):
    # Check if results already exist
    if os.path.exists(results_path):
        print(f"📦 Loading existing results from {results_path}...")
        return pd.read_csv(results_path)

    final_results = []

    for ticker, df in garch_data_dict.items():
        print(f"\n================= {ticker} =================")
        results = train_ml_models_baseline(df, ticker=ticker)

        for model_name, metrics in results.items():
            final_results.append({
                "Stock": ticker,
                "Model": model_name,
                "R2": round(metrics["R2"], 4),
                "RMSE": round(metrics["RMSE"], 4),
                "MSE": round(metrics["MSE"], 6),
                "MAE": round(metrics["MAE"], 4),
            })

    # Save results
    results_df = pd.DataFrame(final_results)
    results_df.to_csv(results_path, index=False)
    print(f"💾 Saved results to {results_path}")

    return results_df

# Run training or load existing results
ml_all_results = train_all_stocks_ml_baseline(garch_data)
ml_all_results_sorted = ml_all_results.sort_values(by="R2", ascending=False)
display(ml_all_results_sorted)


================= HDFCBANK.NS =================

📈 Training ML models for HDFCBANK.NS...
✅ KNN — R²: -2.0136, RMSE: 0.0077, MAE: 0.0068
✅ AdaBoost — R²: 0.8690, RMSE: 0.0016, MAE: 0.0011
✅ CatBoost — R²: 0.8649, RMSE: 0.0016, MAE: 0.0012
✅ XGBoost — R²: 0.8966, RMSE: 0.0014, MAE: 0.0009
✅ RandomForest — R²: 0.9185, RMSE: 0.0013, MAE: 0.0008

================= ICICIBANK.NS =================

📈 Training ML models for ICICIBANK.NS...
✅ KNN — R²: -11.1402, RMSE: 0.0153, MAE: 0.0144
✅ AdaBoost — R²: -0.0277, RMSE: 0.0045, MAE: 0.0037
✅ CatBoost — R²: 0.6083, RMSE: 0.0027, MAE: 0.0022
✅ XGBoost — R²: 0.1282, RMSE: 0.0041, MAE: 0.0028
✅ RandomForest — R²: 0.6080, RMSE: 0.0027, MAE: 0.0019

================= SBIN.NS =================

📈 Training ML models for SBIN.NS...
✅ KNN — R²: -0.3261, RMSE: 0.0042, MAE: 0.0034
✅ AdaBoost — R²: 0.3450, RMSE: 0.0029, MAE: 0.0025
✅ CatBoost — R²: 0.7158, RMSE: 0.0019, MAE: 0.0014
✅ XGBoost — R²: 0.5728, RMSE: 0.0024, MAE: 0.0013
✅ RandomForest — R²: 0.4707

,Stock,Model,R2,RMSE,MSE,MAE
4,HDFCBANK.NS,RandomForest,0.9185,0.0013,0.000002,0.0008
33,INDUSINDBK.NS,XGBoost,0.9159,0.0016,0.000003,0.0011
34,INDUSINDBK.NS,RandomForest,0.9119,0.0017,0.000003,0.0011
32,INDUSINDBK.NS,CatBoost,0.9058,0.0017,0.000003,0.0013
17,KOTAKBANK.NS,CatBoost,0.8980,0.0013,0.000002,0.0009
3,HDFCBANK.NS,XGBoost,0.8966,0.0014,0.000002,0.0009
19,KOTAKBANK.NS,RandomForest,0.8918,0.0013,0.000002,0.0009
18,KOTAKBANK.NS,XGBoost,0.8798,0.0014,0.000002,0.0010
44,LICHSGFIN.NS,RandomForest,0.8786,0.0018,0.000003,0.0012
22,AXISBANK.NS,CatBoost,0.8767,0.0015,0.000002,0.0011


In [8]:
from arch import arch_model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_series(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def forecast_volatility_arch(df, model_type="GARCH", ticker="TICKER", verbose=True):
    df = df.copy()

    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if "Vol_target" not in df.columns or "Return" not in df.columns:
        raise KeyError(f"Missing 'Vol_target' or 'Return' in {ticker}")

    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # fix for scale warning

    if verbose:
        print(f"\n🔮 Running {model_type} for {ticker}...")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor

        try:
            if model_type == "GARCH":
                model = arch_model(train_window, vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            elif model_type == "GJR":
                model = arch_model(train_window, vol='GARCH', p=1, o=1, q=1, dist='normal', rescale=False)
            elif model_type == "EGARCH":
                model = arch_model(train_window, vol='EGARCH', p=1, q=1, dist='normal', rescale=False)
            else:
                raise ValueError("Invalid model_type")

            model_fit = model.fit(disp="off")
            forecast = model_fit.forecast(horizon=1)
            pred_vol = np.sqrt(forecast.variance.values[-1][0]) / scale_factor

        except Exception as e:
            if verbose:
                print(f"⚠️ {model_type} failed at index {i}: {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → {model_type} | index {i} | vol: {pred_vol:.5f}")

    df[f"{model_type}_pred"] = [np.nan] * window_size + preds
    df = df.dropna(subset=["Vol_target", f"{model_type}_pred"])

    metrics = evaluate_series(df["Vol_target"], df[f"{model_type}_pred"])
    if verbose:
        print(f"✅ {model_type} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return df, metrics

In [9]:
import os
import pandas as pd

def evaluate_all_series_models(garch_data_dict, results_path="India_ts_model_results.csv"):
    # If results already exist, load them
    if os.path.exists(results_path):
        print(f"📦 Loading saved time series results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for model_type in ["GARCH", "GJR", "EGARCH"]:
            print(f"\n================= {ticker} - {model_type} =================")
            try:
                _, metrics = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=True)
                results.append({
                    "Stock": ticker,
                    "Model": model_type,
                    "R2": round(metrics["R2"], 4),
                    "RMSE": round(metrics["RMSE"], 4),
                    "MSE": round(metrics["MSE"], 6),
                    "MAE": round(metrics["MAE"], 4),
                })
            except Exception as e:
                print(f"⚠️ Skipping {ticker} - {model_type}: {e}")

    df_results = pd.DataFrame(results)
    df_results.to_csv(results_path, index=False)
    print(f"💾 Saved time series model results to {results_path}")

    return df_results

ts_model_results = evaluate_all_series_models(garch_data)
ts_model_results_sorted = ts_model_results.sort_values(by="R2", ascending=False)
display(ts_model_results_sorted)


================= HDFCBANK.NS - GARCH =================

🔮 Running GARCH for HDFCBANK.NS...
  → GARCH | index 500 | vol: 0.00954
  → GARCH | index 750 | vol: 0.00975
  → GARCH | index 1000 | vol: 0.05400
  → GARCH | index 1250 | vol: 0.02548
  → GARCH | index 1500 | vol: 0.03073
  → GARCH | index 1750 | vol: 0.01220
✅ GARCH for HDFCBANK.NS — R²: 0.6730, RMSE: 0.0043, MAE: 0.0026

================= HDFCBANK.NS - GJR =================

🔮 Running GJR for HDFCBANK.NS...
  → GJR | index 500 | vol: 0.00973
  → GJR | index 750 | vol: 0.00977
  → GJR | index 1000 | vol: 0.05564
  → GJR | index 1250 | vol: 0.02526
  → GJR | index 1500 | vol: 0.02704
  → GJR | index 1750 | vol: 0.01152
✅ GJR for HDFCBANK.NS — R²: 0.5902, RMSE: 0.0049, MAE: 0.0027

================= HDFCBANK.NS - EGARCH =================

🔮 Running EGARCH for HDFCBANK.NS...
  → EGARCH | index 500 | vol: 0.00956
  → EGARCH | index 750 | vol: 0.01008
  → EGARCH | index 1000 | vol: 0.04784
  → EGARCH | index 1250 | vol: 0.02533
  →

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → GJR | index 750 | vol: 0.01428
  → GJR | index 1000 | vol: 0.08117
  → GJR | index 1250 | vol: 0.02285
  → GJR | index 1500 | vol: 0.01971
  → GJR | index 1750 | vol: 0.01380
✅ GJR for ICICIBANK.NS — R²: 0.7145, RMSE: 0.0053, MAE: 0.0036

================= ICICIBANK.NS - EGARCH =================

🔮 Running EGARCH for ICICIBANK.NS...
  → EGARCH | index 500 | vol: 0.02011
  → EGARCH | index 750 | vol: 0.01582
  → EGARCH | index 1000 | vol: 0.05663
  → EGARCH | index 1250 | vol: 0.02397
  → EGARCH | index 1500 | vol: 0.01919


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

  → EGARCH | index 1750 | vol: 0.01369


...

/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/

  → EGARCH | index 1000 | vol: 0.06123
  → EGARCH | index 1250 | vol: 0.02617
  → EGARCH | index 1500 | vol: 0.02408
  → EGARCH | index 1750 | vol: 0.01989
✅ EGARCH for LICHSGFIN.NS — R²: -50811803394610269564306657902592.0000, RMSE: 66958676969591.5000, MAE: 1771914492138.2888

================= HDFCLIFE.NS - GARCH =================

🔮 Running GARCH for HDFCLIFE.NS...
  → GARCH | index 500 | vol: 0.01468
  → GARCH | index 750 | vol: 0.01322
✅ GARCH for HDFCLIFE.NS — R²: -0.1147, RMSE: 0.0053, MAE: 0.0037

================= HDFCLIFE.NS - GJR =================

🔮 Running GJR for HDFCLIFE.NS...
  → GJR | index 500 | vol: 0.01510
  → GJR | index 750 | vol: 0.01281
✅ GJR for HDFCLIFE.NS — R²: -0.0420, RMSE: 0.0051, MAE: 0.0035

================= HDFCLIFE.NS - EGARCH =================

🔮 Running EGARCH for HDFCLIFE.NS...
  → EGARCH | index 500 | vol: 0.01533
  → EGARCH | index 750 | vol: 0.01225


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


✅ EGARCH for HDFCLIFE.NS — R²: -1.2272, RMSE: 0.0074, MAE: 0.0041

================= SBILIFE.NS - GARCH =================

🔮 Running GARCH for SBILIFE.NS...
  → GARCH | index 500 | vol: 0.01493
  → GARCH | index 750 | vol: 0.01243
  → GARCH | index 1000 | vol: 0.01356
✅ GARCH for SBILIFE.NS — R²: 0.3573, RMSE: 0.0032, MAE: 0.0026

================= SBILIFE.NS - GJR =================

🔮 Running GJR for SBILIFE.NS...
  → GJR | index 500 | vol: 0.01552
  → GJR | index 750 | vol: 0.01247
  → GJR | index 1000 | vol: 0.01388
✅ GJR for SBILIFE.NS — R²: 0.3092, RMSE: 0.0033, MAE: 0.0027

================= SBILIFE.NS - EGARCH =================

🔮 Running EGARCH for SBILIFE.NS...
  → EGARCH | index 500 | vol: 0.01621
  → EGARCH | index 750 | vol: 0.01243
  → EGARCH | index 1000 | vol: 0.01363
✅ EGARCH for SBILIFE.NS — R²: 0.5022, RMSE: 0.0028, MAE: 0.0024

================= UTIAMC.NS - GARCH =================

🔮 Running GARCH for UTIAMC.NS...
⚠️ Skipping UTIAMC.NS - GARCH: Length of values (500)

,Stock,Model,R2,RMSE,MSE,MAE
24,LICHSGFIN.NS,GARCH,8.003000e-01,4.200000e-03,1.800000e-05,3.300000e-03
12,AXISBANK.NS,GARCH,7.944000e-01,5.200000e-03,2.800000e-05,3.300000e-03
3,ICICIBANK.NS,GARCH,7.662000e-01,4.800000e-03,2.300000e-05,3.500000e-03
25,LICHSGFIN.NS,GJR,7.598000e-01,4.600000e-03,2.100000e-05,3.600000e-03
4,ICICIBANK.NS,GJR,7.145000e-01,5.300000e-03,2.800000e-05,3.600000e-03
18,INDUSINDBK.NS,GARCH,6.981000e-01,9.600000e-03,9.200000e-05,5.200000e-03
19,INDUSINDBK.NS,GJR,6.906000e-01,9.700000e-03,9.400000e-05,5.400000e-03
0,HDFCBANK.NS,GARCH,6.730000e-01,4.300000e-03,1.900000e-05,2.600000e-03
13,AXISBANK.NS,GJR,6.335000e-01,7.000000e-03,4.900000e-05,3.700000e-03
2,HDFCBANK.NS,EGARCH,6.276000e-01,4.600000e-03,2.100000e-05,2.600000e-03


In [10]:
def train_fusion_model(df, model_name, ts_feature="GARCH_pred", ticker="TICKER"):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    if model_name not in models:
        raise ValueError(f"Model '{model_name}' not recognized.")

    df = df.copy()

    # Flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if ts_feature not in df.columns:
        raise ValueError(f"'{ts_feature}' not found in DataFrame for {ticker}")

    feature_cols = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D', 'Vol_t_1', 'Vol_t_2', 'Vol_t_3',
        'Vol_t_4', 'Vol_t_5', 'Vol_t_6', ts_feature
    ]

    X = df[feature_cols].copy()
    y = df["Vol_target"]

    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = evaluate_series(y_test, y_pred)

    print(f"✅ {model_name} + {ts_feature} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return metrics

In [11]:
import os
import pandas as pd

# Reuse existing forecast function
for ticker in tqdm(garch_data.keys()):
    for model_type in ["GJR", "EGARCH"]:
        print(f"\n📈 Adding {model_type}_pred to {ticker}...")
        df = garch_data[ticker]

        try:
            df, _ = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=False)
            garch_data[ticker] = df  # Update with new column
        except Exception as e:
            print(f"⚠️ {model_type} failed for {ticker}: {e}")

def train_all_fusion_models(garch_data_dict, results_path="India_fusion_model_results.csv"):
    # Load existing results if file exists
    if os.path.exists(results_path):
        print(f"📦 Loading saved fusion model results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for ts_feature in ["GARCH_pred", "GJR_pred", "EGARCH_pred"]:
            if ts_feature not in df.columns:
                print(f"⚠️ Skipping {ticker} - missing {ts_feature}")
                continue

            for model_name in ["RandomForest", "XGBoost", "CatBoost", "AdaBoost", "KNN"]:
                try:
                    metrics = train_fusion_model(df, model_name, ts_feature=ts_feature, ticker=ticker)
                    results.append({
                        "Stock": ticker,
                        "Fusion_Model": f"{ts_feature}+{model_name}",
                        "R2": round(metrics["R2"], 4),
                        "RMSE": round(metrics["RMSE"], 4),
                        "MSE": round(metrics["MSE"], 6),
                        "MAE": round(metrics["MAE"], 4),
                    })
                except Exception as e:
                    print(f"⚠️ {ticker} {ts_feature}+{model_name} failed: {e}")

    fusion_df = pd.DataFrame(results)
    fusion_df.to_csv(results_path, index=False)
    print(f"💾 Saved fusion model results to {results_path}")

    return fusion_df

fusion_results_df = train_all_fusion_models(garch_data)
fusion_results_sorted = fusion_results_df.sort_values(by="R2", ascending=False)
display(fusion_results_sorted)

  0%|          | 0/12 [00:00<?, ?it/s]


📈 Adding GJR_pred to HDFCBANK.NS...

📈 Adding EGARCH_pred to HDFCBANK.NS...


  8%|▊         | 1/12 [00:33<06:05, 33.27s/it]


📈 Adding GJR_pred to ICICIBANK.NS...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


...


📈 Adding EGARCH_pred to HDFCLIFE.NS...
⚠️ EGARCH failed for HDFCLIFE.NS: Length of values (500) does not match length of index (474)

📈 Adding GJR_pred to SBILIFE.NS...


100%|██████████| 12/12 [05:16<00:00, 26.34s/it]


📈 Adding EGARCH_pred to SBILIFE.NS...

📈 Adding GJR_pred to UTIAMC.NS...
⚠️ GJR failed for UTIAMC.NS: Length of values (500) does not match length of index (260)

📈 Adding EGARCH_pred to UTIAMC.NS...
⚠️ EGARCH failed for UTIAMC.NS: Length of values (500) does not match length of index (260)


✅ RandomForest + GARCH_pred for HDFCBANK.NS — R²: 0.6636, RMSE: 0.0012, MAE: 0.0008
✅ XGBoost + GARCH_pred for HDFCBANK.NS — R²: 0.6274, RMSE: 0.0013, MAE: 0.0009
✅ CatBoost + GARCH_pred for HDFCBANK.NS — R²: 0.2466, RMSE: 0.0019, MAE: 0.0014
✅ AdaBoost + GARCH_pred for HDFCBANK.NS — R²: 0.3336, RMSE: 0.0017, MAE: 0.0013
✅ KNN + GARCH_pred for HDFCBANK.NS — R²: -17.7165, RMSE: 0.0092, MAE: 0.0084
✅ RandomForest + GJR_pred for HDFCBANK.NS — R²: 0.6727, RMSE: 0.0012, MAE: 0.0008
✅ XGBoost + GJR_pred for HDFCBANK.NS — R²: 0.4746, RMSE: 0.0015, MAE: 0.0012
✅ CatBoost + GJR_pred for HDFCBANK.NS — R²: 0.1597, RMSE: 0.0020, MAE: 0.0016
✅ AdaBoost + GJR_pred for HDFCBANK.NS — R²: 0.2206, RMSE: 0.0019, MAE: 0.0014
✅ KNN + GJR_pred for HDFCBANK.NS — R²: -17.7165, RMSE: 0.0092, MAE: 0.0084
✅ RandomForest + EGARCH_pred for HDFCBANK.NS — R²: 0.6931, RMSE: 0.0012, MAE: 0.0008
✅ XGBoost + EGARCH_pred for HDFCBANK.NS — R²: 0.5596, RMSE: 0.0014, MAE: 0.0010
✅ CatBoost + EGARCH_pred for HDFCBANK.NS — R²

,Stock,Fusion_Model,R2,RMSE,MSE,MAE
85,PNB.NS,EGARCH_pred+RandomForest,0.8426,0.0021,0.000004,0.0017
90,INDUSINDBK.NS,GARCH_pred+RandomForest,0.8307,0.0014,0.000002,0.0010
75,PNB.NS,GARCH_pred+RandomForest,0.8238,0.0022,0.000005,0.0018
95,INDUSINDBK.NS,GJR_pred+RandomForest,0.8208,0.0014,0.000002,0.0010
40,SBIN.NS,EGARCH_pred+RandomForest,0.7657,0.0015,0.000002,0.0011
...,...,...,...,...,...,...
9,HDFCBANK.NS,GJR_pred+KNN,-17.7165,0.0092,0.000085,0.0084
69,AXISBANK.NS,GJR_pred+KNN,-17.7653,0.0117,0.000137,0.0114
64,AXISBANK.NS,GARCH_pred+KNN,-17.7653,0.0117,0.000137,0.0114
74,AXISBANK.NS,EGARCH_pred+KNN,-17.7653,0.0117,0.000137,0.0114
